In [1]:
import pandas as pd

# Read the data from the "staff.xlsx" file
staff_data = pd.read_excel("staff.xlsx")

# Convert 'Month' column to datetime
staff_data['Month'] = pd.to_datetime(staff_data['Month'])

# Group by 'Job', 'Project', 'Month', and 'Type' and count the occurrences of each group
count_data = staff_data.groupby(['Job', 'Project', 'Month', 'Type']).size().reset_index(name='Count')

# Find the row with the maximum count for each group of 'Job' and 'Project'
peak_values = count_data.loc[count_data.groupby(['Job', 'Project'])['Count'].idxmax()]

# Rename the 'Month' column to 'Peak Month'
peak_values.rename(columns={'Month': 'Peak Month'}, inplace=True)

# Convert 'Peak Month' column to show month-Year format
peak_values['Peak Month'] = peak_values['Peak Month'].dt.strftime('%b-%Y')

# Save the results to a new Excel file
peak_values.to_excel("peak_values.xlsx", index=False)


In [2]:
import pandas as pd

# Read the peak_values.xlsx file
peak_values = pd.read_excel("peak_values.xlsx")

# Group the data by the "Type" column
grouped_by_type = peak_values.groupby('Type')

# Initialize an empty list to store the DataFrames for each group
average_job_counts_by_type_list = []

# Iterate over each group
for group_name, group_data in grouped_by_type:
    # Get the unique jobs in the group
    unique_jobs = group_data['Job'].unique()
    
    # Calculate the average count for each job
    average_counts = []
    for job in unique_jobs:
        avg_count = group_data[group_data['Job'] == job]['Count'].mean()
        average_counts.append(avg_count)
    
    # Create a DataFrame with the results for this group
    group_results = pd.DataFrame({
        'Type': [group_name] * len(unique_jobs),
        'Job': unique_jobs,
        'Average Count': average_counts
    })
    
    # Append the group results to the list
    average_job_counts_by_type_list.append(group_results)

# Concatenate all DataFrames in the list into one DataFrame
average_job_counts_by_type = pd.concat(average_job_counts_by_type_list, ignore_index=True)

# Save the results to a new Excel file
average_job_counts_by_type.to_excel("average_job_counts_by_type.xlsx", index=False)


In [3]:
import pandas as pd

# Take input from the user for the type of project and its duration
project_type = input("Enter the type of project: ")
duration = int(input("Enter the duration of the project (in months): "))

# Determine the start month (Mar-24) and end month based on the duration
start_month = pd.to_datetime('2024-03-01')
end_month = start_month + pd.DateOffset(months=duration - 1)

# Create a list of months from start to end
months = pd.date_range(start=start_month, end=end_month, freq='MS')

# Initialize the deployment plan DataFrame
deployment_plan = pd.DataFrame({'Month': months})

# Calculate the middle month index (for the start of the average period)
middle_month_index = len(deployment_plan) // 2

# Define the pattern for increasing and decreasing counts
increasing_pattern = list(range(middle_month_index))
decreasing_pattern = list(range(middle_month_index, 0, -1))

# Calculate the number of people for each job in each month
for job in average_job_counts_by_type['Job']:
    # Calculate the average count for the job
    avg_count = average_job_counts_by_type.loc[average_job_counts_by_type['Job'] == job, 'Average Count'].iloc[0]
    
    # Initialize counts list for the job
    counts = []
    
    # Generate counts based on the pattern
    for i in range(len(deployment_plan)):
        if i < middle_month_index:
            counts.append(int(round((increasing_pattern[i] / middle_month_index) * avg_count)))
        elif i == middle_month_index:
            counts.append(int(avg_count))
        else:
            counts.append(int(round((decreasing_pattern[i - middle_month_index] / middle_month_index) * avg_count)))
    
    # Add counts to the deployment plan DataFrame
    deployment_plan[job] = counts

# Output the deployment plan
print("Deployment Plan:")
print(deployment_plan)


Deployment Plan:
        Month  Accounting  Admin  Construction Manager  Control  Driver  \
0  2024-03-01           0      0                     0        0       0   
1  2024-04-01           0      0                     0        0       1   
2  2024-05-01           0      0                     1        0       1   
3  2024-06-01           1      0                     1        1       2   
4  2024-07-01           1      1                     1        1       3   
5  2024-08-01           1      1                     2        1       3   
6  2024-09-01           1      1                     1        1       3   
7  2024-10-01           1      1                     2        1       3   
8  2024-11-01           1      1                     1        1       3   
9  2024-12-01           1      0                     1        1       2   
10 2025-01-01           0      0                     1        0       1   
11 2025-02-01           0      0                     0        0       1   

    Equ